In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tqdm
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import numpy as np

from torch.utils.data import Dataset, DataLoader
from resnet_classifier import load_resnet_classifier

In [2]:
device = "cuda:0"

#model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True).to(device)
#model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=False).to(device)
#model.classifier[1] = nn.Linear(1280, 2).to(device)

#model = load_classifier("FFHQ-Gender_res64.pth", 0, 2)
#model = load_classifier("CelebA-64-nodataaug.pt", 0, 2)

In [3]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True).to(device)
model.fc = nn.Linear(512, 2).to(device)

#model = load_resnet_classifier("resnet-18-64px-unfreezel4.pt", 0, 2)

Using cache found in C:\Users\noahv/.cache\torch\hub\pytorch_vision_v0.10.0


In [4]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
from torchvision.transforms import InterpolationMode
import torch.utils.data as data
import os
from PIL import Image
import pandas as pd
from torchvision import transforms
from torchvision.transforms import functional as vision_F

class FFHQ(data.Dataset):
    def __init__(self, ffhq_dir, csv_path, image_size=32, transform=None, label="gender"):
        """
        PyTorch DataSet for the FFHQ-Age dataset.
        :param root: Root folder that contains a directory for the dataset and the csv with labels in the root directory.
        :param label: Label we want to train on, chosen from the csv labels list.
        """
        self.target_class = label

        # Store image paths
        self.images = [os.path.join(ffhq_dir, file)
                       for file in os.listdir(ffhq_dir) if file.endswith('.jpg')]

        # Import labels from a CSV file
        self.labels = pd.read_csv(csv_path)

        def transform_with_resize(tensor_images):
            return vision_F.resize(tensor_images, [224, 224])

        # Image transformation
        self.transform = transform
        if self.transform is None:
            self.transform = transforms.Compose([
                transforms.Resize(image_size),
                #transforms.Resize(224),
                transforms.ToTensor(),
                #transforms.Resize(224),
                transforms.Lambda(lambda x: transform_with_resize(x)),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])

        # Make a lookup dictionary for the labels
        # Get column names of dataframe
        cols = self.labels.columns.values
        label_ids = {col_name: i for i, col_name in enumerate(cols)}
        self.class_id = label_ids[self.target_class]

        self.one_hot_encoding = {"male": 0,
                                 "female": 1}

    def set_transform(self, transform):
        self.transform = transform

    def __getitem__(self, index):
        _img = self.transform(Image.open(self.images[index]))
        _label = self.one_hot_encoding[self.labels.iloc[index, self.class_id]]
        return _img, _label

    def __len__(self):
        return len(self.images)

In [6]:
import torch.utils.data as data
import os
from PIL import Image
import pandas as pd
from torchvision import transforms

class CelebA(data.Dataset):
    def __init__(self, celeb_dir, csv_path, image_size=32, transform=None, label="male"):
        """
        PyTorch DataSet for the FFHQ-Age dataset.
        :param root: Root folder that contains a directory for the dataset and the csv with labels in the root directory.
        :param label: Label we want to train on, chosen from the csv labels list.
        """
        self.target_class = label

        # Store image paths
        image_path = os.path.join(celeb_dir, "img_align_celeba", "img_align_celeba")
        self.images = [os.path.join(image_path, file)
                       for file in os.listdir(image_path) if file.endswith('.jpg')]

        # Import labels from a CSV file
        self.labels = pd.read_csv(csv_path)

        # Image transformation
        self.transform = transform
        if self.transform is None:
            self.transform = transforms.Compose([
                transforms.Resize(image_size),
                transforms.Resize(224),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])

        # Make a lookup dictionary for the labels
        # Get column names of dataframe
        cols = self.labels.columns.values
        label_ids = {col_name: i for i, col_name in enumerate(cols)}
        self.class_id = label_ids[self.target_class]

    def set_transform(self, transform):
        self.transform = transform

    def __getitem__(self, index):
        _img = self.transform(Image.open(self.images[index]))
        _label = 0 if self.labels.iloc[index, self.class_id] == 1 else 1  # Male will be the first number as with FFHQ upstairs
        return _img, _label

    def __len__(self):
        return len(self.images)

In [8]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

def get_train_valid_test_dataset(celeba_dir, csv_path, label, image_size=32, valid_ratio=0.15, test_ratio=0.15):
    # TODO: Specify different training routines here per class (such as random crop, random horizontal flip, etc.)

    dataset = CelebA(celeba_dir, csv_path, image_size=image_size, label=label)
    train_length, valid_length, test_length = int(len(dataset) * (1 - valid_ratio - test_ratio)), \
                                              int(len(dataset) * valid_ratio), int(len(dataset) * test_ratio)
    # Make sure that the lengths sum to the total length of the dataset
    remainder = len(dataset) - train_length - valid_length - test_length
    train_length += remainder
    train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset,
                                                                             [train_length, valid_length, test_length],
                                                                             generator=torch.Generator().manual_seed(42)
                                                                             )
    """
    train_dataset.set_transform = A.Compose(
        [
            transforms.Resize(image_size),
            A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
            A.HorizontalFlip(p=0.2),
            A.RandomBrightnessContrast(p=0.3, brightness_limit=0.25, contrast_limit=0.5),
            A.MotionBlur(p=.2),
            A.GaussNoise(p=.2),
            A.ImageCompression(p=.2, quality_lower=50),
            A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
            transforms.Resize(224),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ]
    )
    """

    return train_dataset, val_dataset, test_dataset

In [12]:
celeb_train, celeb_val, celeb_test = get_train_valid_test_dataset("./celeba_data", "./celeba_data/list_attr_celeba.csv", "Young", image_size=64)

In [13]:
torch.cuda.empty_cache()
batch_size = 128
cel_train_loader = DataLoader(celeb_train, batch_size=batch_size, pin_memory=True)
cel_val_loader = DataLoader(celeb_val, batch_size=batch_size)
cel_test_loader = DataLoader(celeb_test, batch_size=batch_size)

In [14]:
from tqdm import notebook

optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

model.requires_grad_(False)
model.fc.requires_grad_(True)

def validate_model(model, loader, criterion):
    """Validate the model"""

    # Set the model to evaluation mode.
    model.eval()

    # Initialize the loss and accuracy.
    loss = 0
    accuracy = 0

    # For each batch in the validation set...
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(notebook.tqdm_notebook(loader)):
            # Send the batch to the device.

            data, target = data.to(device), target.to(device)

            # Forward pass.
            output = model(data)

            # Calculate the loss.
            loss += criterion(output, target).item() * len(target)/128

            # Get the predictions.
            preds = torch.argmax(output, 1)

            # Calculate the accuracy.
            accuracy += torch.sum(preds == target).item() * len(target)/128

    # Calculate the average loss and accuracy.
    loss /= len(loader)
    accuracy /= len(loader) * batch_size

    return loss, accuracy


def train_model(model, train_loader, val_loader, optimizer, criterion, test_model=False, epochs=10):
    """Trains model"""

    # Put the model in training mode.
    model.train()

    # For each epoch...
    for epoch in range(epochs):
        # For each batch in the training set...
        for batch_idx, (data, target) in enumerate(notebook.tqdm_notebook(train_loader)):
            # Send the data and labels to the device.
            data, target = data.to(device), target.to(device)

            # Zero out the gradients.
            optimizer.zero_grad()

            # Forward pass.
            output = model(data)

            # Calculate the loss.
            loss = criterion(output, target)

            # Backward pass.
            loss.backward()

            # Update the weights.
            optimizer.step()

            # Print the loss.
            if batch_idx % 100 == 0:
                print('Epoch: {}/{}'.format(epoch + 1, epochs),
                      'Loss: {:.4f}'.format(loss.item()))

        # Validate the model.
        val_loss, val_acc = validate_model(model, val_loader, criterion)

        # Print the validation loss.
        print('Validation Loss: {:.4f}'.format(val_loss))

        # Print the validation accuracy.
        print('Validation Accuracy: {:.4f}'.format(val_acc))

    if test_model:
        # Test the model.
        test_loss, test_acc = validate_model(model, test_loader, criterion)

        # Print the test loss.
        print('Test Loss: {:.4f}'.format(test_loss))

        # Print the test accuracy.
        print('Test Accuracy: {:.4f}'.format(test_acc))

In [15]:
# Trained with only classifier unfrozen
#model.features.requires_grad_(True)
#model.classifier.requires_grad_(True)
train_model(model, cel_train_loader, cel_val_loader, optimizer, criterion, epochs=1)

  0%|          | 0/1108 [00:00<?, ?it/s]

Epoch: 1/1 Loss: 0.9161
Epoch: 1/1 Loss: 0.5585
Epoch: 1/1 Loss: 0.5584
Epoch: 1/1 Loss: 0.5151
Epoch: 1/1 Loss: 0.4459
Epoch: 1/1 Loss: 0.3432
Epoch: 1/1 Loss: 0.4166
Epoch: 1/1 Loss: 0.3557
Epoch: 1/1 Loss: 0.3853
Epoch: 1/1 Loss: 0.3855
Epoch: 1/1 Loss: 0.3347
Epoch: 1/1 Loss: 0.4234


  0%|          | 0/238 [00:00<?, ?it/s]

Validation Loss: 0.4009
Validation Accuracy: 0.8213


In [16]:
model.layer4.requires_grad_(True)
train_model(model, cel_train_loader, cel_val_loader, optimizer, criterion, epochs=1)

  0%|          | 0/1108 [00:00<?, ?it/s]

Epoch: 1/1 Loss: 0.4180
Epoch: 1/1 Loss: 0.2725
Epoch: 1/1 Loss: 0.4427
Epoch: 1/1 Loss: 0.3420
Epoch: 1/1 Loss: 0.2328
Epoch: 1/1 Loss: 0.2085
Epoch: 1/1 Loss: 0.3400
Epoch: 1/1 Loss: 0.2614
Epoch: 1/1 Loss: 0.2582
Epoch: 1/1 Loss: 0.3162
Epoch: 1/1 Loss: 0.2103
Epoch: 1/1 Loss: 0.2890


  0%|          | 0/238 [00:00<?, ?it/s]

Validation Loss: 0.2911
Validation Accuracy: 0.8782


In [17]:
model.layer3.requires_grad_(True)
train_model(model, cel_train_loader, cel_val_loader, optimizer, criterion, epochs=1)

  0%|          | 0/1108 [00:00<?, ?it/s]

Epoch: 1/1 Loss: 0.2447
Epoch: 1/1 Loss: 0.2122
Epoch: 1/1 Loss: 0.3601
Epoch: 1/1 Loss: 0.2718
Epoch: 1/1 Loss: 0.1609
Epoch: 1/1 Loss: 0.1964
Epoch: 1/1 Loss: 0.3073
Epoch: 1/1 Loss: 0.2261
Epoch: 1/1 Loss: 0.2123
Epoch: 1/1 Loss: 0.2652
Epoch: 1/1 Loss: 0.1740
Epoch: 1/1 Loss: 0.2032


  0%|          | 0/238 [00:00<?, ?it/s]

Validation Loss: 0.2845
Validation Accuracy: 0.8800


In [18]:
# Test the model.
test_loss, test_acc = validate_model(model, cel_test_loader, criterion)

# Print the test loss.
print('Test Loss: {:.4f}'.format(test_loss))

# Print the test accuracy.
print('Test Accuracy: {:.4f}'.format(test_acc))

  0%|          | 0/238 [00:00<?, ?it/s]

Test Loss: 0.2846
Test Accuracy: 0.8778


In [29]:
# Freeze/unfreeze layers after converging with training

#model.features[0:15].requires_grad_(False)
#model.features[15:].requires_grad_(True)

Sequential(
  (15): InvertedResidual(
    (conv): Sequential(
      (0): ConvNormActivation(
        (0): Conv2d(160, 960, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(960, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): ConvNormActivation(
        (0): Conv2d(960, 960, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=960, bias=False)
        (1): BatchNorm2d(960, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (2): Conv2d(960, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (3): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (16): InvertedResidual(
    (conv): Sequential(
      (0): ConvNormActivation(
        (0): Conv2d(160, 960, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(960, eps=1e-05, momentum=0.1, affine=True, track_running_stats=T

In [30]:
# Trained with layers until 15 frozen
train_model(model, train_loader, val_loader, optimizer, criterion, epochs=1)

  0%|          | 0/1108 [00:00<?, ?it/s]

Epoch: 1/1 Loss: 0.2442
Epoch: 1/1 Loss: 0.0951
Epoch: 1/1 Loss: 0.0936
Epoch: 1/1 Loss: 0.1011
Epoch: 1/1 Loss: 0.0920
Epoch: 1/1 Loss: 0.2772
Epoch: 1/1 Loss: 0.0893
Epoch: 1/1 Loss: 0.1182
Epoch: 1/1 Loss: 0.0601
Epoch: 1/1 Loss: 0.0659
Epoch: 1/1 Loss: 0.1026
Epoch: 1/1 Loss: 0.1404


  0%|          | 0/238 [00:00<?, ?it/s]

Validation Loss: 0.0667
Validation Accuracy: 0.9716


  0%|          | 0/238 [00:00<?, ?it/s]

Test Loss: 0.0664
Test Accuracy: 0.9719


In [31]:
# Freeze/unfreeze layers after converging with training

model.features[0:13].requires_grad_(False)
model.features[13:].requires_grad_(True)

Sequential(
  (13): InvertedResidual(
    (conv): Sequential(
      (0): ConvNormActivation(
        (0): Conv2d(96, 576, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(576, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): ConvNormActivation(
        (0): Conv2d(576, 576, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=576, bias=False)
        (1): BatchNorm2d(576, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (2): Conv2d(576, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (3): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (14): InvertedResidual(
    (conv): Sequential(
      (0): ConvNormActivation(
        (0): Conv2d(96, 576, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(576, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [32]:
# Trained with layers until 13 frozen
train_model(model, train_loader, val_loader, optimizer, criterion, epochs=1)

  0%|          | 0/1108 [00:00<?, ?it/s]

Epoch: 1/1 Loss: 0.0420
Epoch: 1/1 Loss: 0.0411
Epoch: 1/1 Loss: 0.0187
Epoch: 1/1 Loss: 0.0586
Epoch: 1/1 Loss: 0.0439
Epoch: 1/1 Loss: 0.1894
Epoch: 1/1 Loss: 0.0527
Epoch: 1/1 Loss: 0.0850
Epoch: 1/1 Loss: 0.0492
Epoch: 1/1 Loss: 0.0377
Epoch: 1/1 Loss: 0.0611
Epoch: 1/1 Loss: 0.0718


  0%|          | 0/238 [00:00<?, ?it/s]

Validation Loss: 0.0562
Validation Accuracy: 0.9758


  0%|          | 0/238 [00:00<?, ?it/s]

Test Loss: 0.0552
Test Accuracy: 0.9766


In [162]:
# Only for Resnet, train with all layers unfrozen
import gc
torch.cuda.empty_cache()
gc.collect()
model.requires_grad_(True)
train_model(model, train_loader, val_loader, optimizer, criterion, epochs=1)

  0%|          | 0/4432 [00:00<?, ?it/s]

Epoch: 1/1 Loss: 0.3006
Epoch: 1/1 Loss: 0.0938
Epoch: 1/1 Loss: 0.0645


KeyboardInterrupt: 

In [163]:
# Test the model.
test_loss, test_acc = validate_model(model, test_loader, criterion)

# Print the test loss.
print('Test Loss: {:.4f}'.format(test_loss))

# Print the test accuracy.
print('Test Accuracy: {:.4f}'.format(test_acc))

  0%|          | 0/950 [00:00<?, ?it/s]

Test Loss: 0.0249
Test Accuracy: 0.2408


In [19]:
# Save model state dict to file

# Trained on 256 works better than 128 for 128
torch.save(model.state_dict(), './resnet-18-64px-age-classifier.pt')

In [32]:
#_train_loader = DataLoader(train, batch_size=128)

In [46]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

def get_train_valid_test_dataset(ffhq_dir, csv_path, label, image_size=32, valid_ratio=0.15, test_ratio=0.15):
    # TODO: Specify different training routines here per class (such as random crop, random horizontal flip, etc.)

    dataset = FFHQ(ffhq_dir, csv_path, image_size=image_size, label=label)
    train_length, valid_length, test_length = int(len(dataset) * (1 - valid_ratio - test_ratio)), \
                                              int(len(dataset) * valid_ratio), int(len(dataset) * test_ratio)
    # Make sure that the lengths sum to the total length of the dataset
    remainder = len(dataset) - train_length - valid_length - test_length
    train_length += remainder
    train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset,
                                                                             [train_length, valid_length, test_length],
                                                                             generator=torch.Generator().manual_seed(42)
                                                                             )
    """
    train_dataset.set_transform = A.Compose(
        [
            transforms.Resize(image_size),
            A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
            A.HorizontalFlip(p=0.2),
            A.RandomBrightnessContrast(p=0.3, brightness_limit=0.25, contrast_limit=0.5),
            A.MotionBlur(p=.2),
            A.GaussNoise(p=.2),
            A.ImageCompression(p=.2, quality_lower=50),
            A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
            transforms.Resize(224),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ]
    )
    """

    return train_dataset, val_dataset, test_dataset

In [47]:
train, val, test = get_train_valid_test_dataset("./data", "./ffhq_aging_labels.csv", "gender", image_size=128)
train_loader = DataLoader(train, batch_size=128)
val_loader = DataLoader(val, batch_size=128)
test_loader = DataLoader(test, batch_size=128)

In [48]:
from sklearn.metrics import confusion_matrix
from tqdm import notebook

y_preds = []
y_trues = []

model.eval()

with torch.no_grad():
    for i, (data, target) in enumerate(notebook.tqdm_notebook(train_loader)):
        y_trues.append(target.cpu())

        data, target = data.to(device), target.to(device)
        output = model(data)

        preds = torch.argmax(output, 1)

        y_preds.append(preds.cpu())

  0%|          | 0/383 [00:00<?, ?it/s]

In [49]:
from sklearn.metrics import confusion_matrix
from tqdm import notebook

#y_preds = []
#y_trues = []

model.eval()

with torch.no_grad():
    for i, (data, target) in enumerate(notebook.tqdm_notebook(val_loader)):
        y_trues.append(target.cpu())

        data, target = data.to(device), target.to(device)
        output = model(data)

        preds = torch.argmax(output, 1)

        y_preds.append(preds.cpu())


  0%|          | 0/83 [00:00<?, ?it/s]

In [50]:
from sklearn.metrics import confusion_matrix
from tqdm import notebook

#y_preds = []
#y_trues = []

model.eval()

with torch.no_grad():
    for i, (data, target) in enumerate(notebook.tqdm_notebook(test_loader)):
        y_trues.append(target.cpu())

        data, target = data.to(device), target.to(device)
        output = model(data)

        preds = torch.argmax(output, 1)

        y_preds.append(preds.cpu())

  0%|          | 0/83 [00:00<?, ?it/s]

In [51]:
y_preds = np.concatenate(y_preds)
y_trues = np.concatenate(y_trues)

In [52]:
C = confusion_matrix(y_trues, y_preds)

In [53]:
print("Entire dataset:")
C

Entire dataset:


array([[28018,  4152],
       [ 2494, 35336]], dtype=int64)

In [54]:
from sklearn.metrics import accuracy_score
accuracy_score(y_trues, y_preds)

0.9050571428571429

In [33]:
# Trying to figure out channel normalization
import torch

images = torch.rand((5, 32, 3))

In [39]:
batch_size = 2

In [46]:
samples = torch.distributions.uniform.Uniform(0, 1).sample([batch_size, 1])
torch.cat((torch.ones(batch_size, 1), torch.zeros(batch_size, 1)), dim=1) - torch.cat((samples, torch.zeros((batch_size, 1))), dim=1)

tensor([[0.9722, 0.0000],
        [0.3404, 0.0000]])

In [48]:
samples = torch.distributions.uniform.Uniform(0, 1).sample([batch_size, 1])
torch.cat((torch.ones(batch_size, 1), torch.zeros(batch_size, 1)), dim=1) - torch.cat((samples, torch.zeros((batch_size, 1))), dim=1) + torch.cat((torch.zeros(batch_size, 1), samples), dim=1)

tensor([[0.7501, 0.2499],
        [0.5301, 0.4699]])